# Problem Set 4 - 4
### (a)

In [71]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [72]:
# read dta data
data = pd.read_stata('SAVEWORD.DTA')  # Unfortunatly, the original name of this file is so sensitive that Auto-completion rejects to work with it.
data.head().T


,0,1,2,3,4
id,1,1,1,2,2
state,AL,AL,AL,AK,AK
year,87,90,93,87,90
mrdrte,9.3,11.6,11.6,10.1,7.5
exec,2,5,2,0,0
unem,7.8,6.8,7.5,10.8,6.9
d90,0,1,0,0,1
d93,0,0,1,0,0
cmrdrte,NaN,2.3,0.0,NaN,-2.6
cexec,NaN,3.0,-3.0,NaN,0.0


In [73]:
# create dummies for each year, drop 1987
df = pd.concat([data, pd.get_dummies(data['year'], drop_first=True).astype(float)], axis=1)

# rename the dummies
df.rename(columns={90: 'year_90', 93: 'year_93'}, inplace=True)
df.head().T

,0,1,2,3,4
id,1,1,1,2,2
state,AL,AL,AL,AK,AK
year,87,90,93,87,90
mrdrte,9.3,11.6,11.6,10.1,7.5
exec,2,5,2,0,0
unem,7.8,6.8,7.5,10.8,6.9
d90,0,1,0,0,1
d93,0,0,1,0,0
cmrdrte,NaN,2.3,0.0,NaN,-2.6
cexec,NaN,3.0,-3.0,NaN,0.0


In [74]:
# run regression of mrdrte on the created dummies, exec, unem and a constant
reg = smf.ols('mrdrte ~ exec + unem + year_90 + year_93', data=df).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     3.047
Date:                Mon, 04 Dec 2023   Prob (F-statistic):             0.0190
Time:                        21:23:57   Log-Likelihood:                -549.96
No. Observations:                 153   AIC:                             1110.
Df Residuals:                     148   BIC:                             1125.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8644      3.070     -0.607      0.5

### (b)

In [75]:
# declare panel data with year and id as index
df_panel = df.set_index(['id', 'year'])
df_panel['const'] = 1
df_panel.head().T

id         1                 2     
year      87    90    93    87   90
state     AL    AL    AL    AK   AK
mrdrte   9.3  11.6  11.6  10.1  7.5
exec       2     5     2     0    0
unem     7.8   6.8   7.5  10.8  6.9
d90        0     1     0     0    1
d93        0     0     1     0    0
cmrdrte  NaN   2.3   0.0   NaN -2.6
cexec    NaN   3.0  -3.0   NaN  0.0
cunem    NaN  -1.0   0.7   NaN -3.9
cexec_1  NaN   NaN   3.0   NaN  NaN
cunem_1  NaN   NaN  -1.0   NaN  NaN
year_90  0.0   1.0   0.0   0.0  1.0
year_93  0.0   0.0   1.0   0.0  0.0
const      1     1     1     1    1

In [76]:
# run Fixed Effects regression
from linearmodels import PanelOLS

mod = PanelOLS(df_panel['mrdrte'], df_panel[['exec','unem', 'year_90', 'year_93', 'const']], entity_effects=True, time_effects=False)
res = mod.fit()
print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 mrdrte   R-squared:                        0.0734
Estimator:                   PanelOLS   R-squared (Between):              0.0037
No. Observations:                 153   R-squared (Within):               0.0734
Date:                Mon, Dec 04 2023   R-squared (Overall):              0.0108
Time:                        21:23:57   Log-likelihood                   -375.63
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1.9398
Entities:                          51   P-value                           0.1098
Avg Obs:                       3.0000   Distribution:                    F(4,98)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             1.9398
                            

Compared to (a), the coefficient of unem is very different (not significant anymore) which suggests that unem is corelated  state fixed effects.


In [77]:
# Save the coefficients of the fixed effects regression
beta_fe = res.params

# save the covariance matrix of the fixed effects regression
V_fe = res.cov
sd_fe = res.std_errors

print(beta_fe)
print(V_fe)
print(sd_fe)

exec      -0.138323
unem       0.221316
year_90    1.556215
year_93    1.733242
const      5.822104
Name: parameter, dtype: float64
             exec      unem   year_90   year_93     const
exec     0.031331 -0.004388  0.004712 -0.011621 -0.009985
unem    -0.004388  0.087839  0.076644  0.000601 -0.545034
year_90  0.004712  0.076644  0.555513  0.240489 -0.728936
year_93 -0.011621  0.000601  0.240489  0.490614 -0.233014
const   -0.009985 -0.545034 -0.728936 -0.233014  3.669566
exec       0.177006
unem       0.296376
year_90    0.745327
year_93    0.700438
const      1.915611
Name: std_error, dtype: float64


### (c)

In [78]:
# create dummy for each state from df
df_state = pd.concat([df, pd.get_dummies(df['id'], drop_first=True).astype(float)], axis=1)
df_state['const'] = 1
df_state.head().T

,0,1,2,3,4
id,1,1,1,2,2
state,AL,AL,AL,AK,AK
year,87,90,93,87,90
mrdrte,9.3,11.6,11.6,10.1,7.5
exec,2,5,2,0,0
...,...,...,...,...,...
48,0.0,0.0,0.0,0.0,0.0
49,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0


In [79]:
for variable in ['mrdrte', 'exec', 'unem', 'year_90', 'year_93']:
    # run regression of mrdrte on the created dummies
    reg_state = sm.OLS(endog=df_state[variable], exog=df_state[[x for x in range(2, 52)] + ['const']]).fit()
    # save the residuals
    df_state['r_{}'.format(variable)]= reg_state.resid

df_state.head().T

,0,1,2,3,4
id,1,1,1,2,2
state,AL,AL,AL,AK,AK
year,87,90,93,87,90
mrdrte,9.3,11.6,11.6,10.1,7.5
exec,2,5,2,0,0
...,...,...,...,...,...
r_mrdrte,-1.533333,0.766667,0.766667,1.233334,-1.366667
r_exec,-1.0,2.0,-1.0,-0.0,-0.0
r_unem,0.433333,-0.566667,0.133333,2.366667,-1.533333
r_year_90,-0.333333,0.666667,-0.333333,-0.333333,0.666667


In [80]:
# run regression of r_mrdrte on r_exec, r_unem, r_year_90, r_year_93 , no constant
reg_within = sm.OLS(endog=df_state['r_mrdrte'], exog=df_state[['r_exec', 'r_unem', 'r_year_90', 'r_year_93']]).fit()
print(reg_within.summary())

                                 OLS Regression Results                                
Dep. Variable:               r_mrdrte   R-squared (uncentered):                   0.073
Model:                            OLS   Adj. R-squared (uncentered):              0.048
Method:                 Least Squares   F-statistic:                              2.949
Date:                Mon, 04 Dec 2023   Prob (F-statistic):                      0.0221
Time:                        21:23:57   Log-Likelihood:                         -375.63
No. Observations:                 153   AIC:                                      759.3
Df Residuals:                     149   BIC:                                      771.4
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [81]:
# compare the coefficients of the fixed effects regression and the regression of r_mrdrte on r_exec, r_unem, r_year_90, r_year_93
print(reg_within.params)
print(beta_fe)

print(reg_within.cov_params())
print(V_fe)



r_exec      -0.138323
r_unem       0.221316
r_year_90    1.556215
r_year_93    1.733242
dtype: float64
exec      -0.138323
unem       0.221316
year_90    1.556215
year_93    1.733242
const      5.822104
Name: parameter, dtype: float64
             r_exec    r_unem  r_year_90  r_year_93
r_exec     0.020607 -0.002886   0.003099  -0.007643
r_unem    -0.002886  0.057773   0.050410   0.000396
r_year_90  0.003099  0.050410   0.365371   0.158174
r_year_93 -0.007643  0.000396   0.158174   0.322685
             exec      unem   year_90   year_93     const
exec     0.031331 -0.004388  0.004712 -0.011621 -0.009985
unem    -0.004388  0.087839  0.076644  0.000601 -0.545034
year_90  0.004712  0.076644  0.555513  0.240489 -0.728936
year_93 -0.011621  0.000601  0.240489  0.490614 -0.233014
const   -0.009985 -0.545034 -0.728936 -0.233014  3.669566


In [82]:
# Adjust the standard errors of the regression 
adjusted_se = np.sqrt((153-4)/(153-4-51)) * reg_within.bse
print(adjusted_se)
print(sd_fe)

r_exec       0.177006
r_unem       0.296376
r_year_90    0.745327
r_year_93    0.700438
dtype: float64
exec       0.177006
unem       0.296376
year_90    0.745327
year_93    0.700438
const      1.915611
Name: std_error, dtype: float64


In fact, we did a within regression in (c). The results of coefficients are the same as those in (b). However, the standard errors are different. After adjusting the standard errors by the degree of freedom, the standard errors are the same.
### (d)

In [83]:
from linearmodels.panel import RandomEffects

# run Random Effects regression
reg_re = RandomEffects(df_panel['mrdrte'], df_panel[['exec','unem', 'year_90', 'year_93', 'const']])
res_re = reg_re.fit()
print(res_re)

                        RandomEffects Estimation Summary                        
Dep. Variable:                 mrdrte   R-squared:                        0.0548
Estimator:              RandomEffects   R-squared (Between):              0.0279
No. Observations:                 153   R-squared (Within):               0.0679
Date:                Mon, Dec 04 2023   R-squared (Overall):              0.0320
Time:                        21:23:58   Log-likelihood                   -407.52
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.1454
Entities:                          51   P-value                           0.0780
Avg Obs:                       3.0000   Distribution:                   F(4,148)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             2.1454
                            

In [84]:
# record the coefficients and covariance matrix of the random effects regression
beta_re = res_re.params
V_re = res_re.cov

# Hausman test
q = np.array(beta_fe[:-1] - beta_re[:-1])
V = np.array(np.array(V_fe)[:-1, :-1] - np.array(V_re)[:-1, :-1])

T = np.matmul(np.matmul(q.T, np.linalg.inv(V)), q)

import scipy.stats as stats
print('Should we reject the null hypothsis?', T>stats.chi2(4).ppf(0.95))

Should we reject the null hypothsis? False


Hausman statistic is smaller than $\chi^2(4)=9.4877$, so we cannot reject the null hypothesis of the specification of Random Effects.